In [1]:
sys.path.insert(0, "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/")
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
from copy import deepcopy

In [3]:
import numpy as np
np.set_printoptions(precision=5, suppress=True)
# try:
#     from tqdm.auto import tqdm
# except ModuleNotFoundError:
#     from tqdm import tqdm
from tqdm import tqdm
import torch
from torch import nn
from torch import optim
from torch import Tensor
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel as DDP, DataParallel as DP
from tensorboardX import SummaryWriter
from easydict import EasyDict as ED
import biosppy.signals.ecg as BSE

In [4]:
from train.train_unet_cpsc2019.train import train

In [5]:
from train.train_unet_cpsc2019.utils import (
    init_logger, get_date_str, dict_to_str, str2bool,
    mask_to_intervals,
)
from train.train_unet_cpsc2019.model import ECG_SUBTRACT_UNET_CPSC2019, ECG_UNET_CPSC2019

In [6]:
from cfg import ModelCfg, TrainCfg

In [7]:
ModelCfg

{'skip_dist': 250,
 'torch_dtype': 'float',
 'fs': 500,
 'spacing': 2.0,
 'classes': ['N'],
 'n_leads': 1,
 'model_name': 'subtract_unet',
 'subtract_unet': {'fs': 500,
  'classes': ['N'],
  'n_leads': 1,
  'skip_dist': 250,
  'torch_dtype': 'float',
  'groups': 1,
  'init_batch_norm': False,
  'init_num_filters': 16,
  'init_filter_length': 21,
  'init_dropouts': [0.0, 0.15, 0.0],
  'batch_norm': True,
  'kernel_initializer': 'he_normal',
  'kw_initializer': {},
  'activation': 'relu',
  'kw_activation': {'inplace': True},
  'down_up_block_num': 3,
  'down_mode': 'max',
  'down_scales': [10, 5, 2],
  'down_num_filters': [[24, 24, 24], [48, 48, 48]],
  'down_filter_lengths': [11, 5],
  'down_dropouts': [[0.0, 0.15, 0.0], [0.0, 0.15, 0.0]],
  'bottom_num_filters': [[96, 96], [96, 96]],
  'bottom_filter_lengths': [[5, 5], [5, 5]],
  'bottom_dilations': [[1, 1], [10, 10]],
  'bottom_dropouts': [[0.15, 0.0], [0.15, 0.0]],
  'up_mode': 'nearest',
  'up_scales': [2, 5, 10],
  'up_num_filters

In [8]:
TrainCfg.db_dir = "/home/wenh06/Jupyter/data/CPSC2019/"
TrainCfg

{'fs': 500,
 'db_dir': '/home/wenh06/Jupyter/data/CPSC2019/',
 'log_dir': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/log',
 'checkpoints': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/checkpoints',
 'model_dir': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/saved_models',
 'final_model_name': None,
 'keep_checkpoint_max': 20,
 'train_ratio': 0.8,
 'input_len': 5000,
 'classes': ['N'],
 'n_leads': 1,
 'bias_thr': 37,
 'skip_dist': 250,
 'label_smoothing': 0.0,
 'random_normalize': True,
 'random_normalize_mean': [-0.05, 0.1],
 'random_normalize_std': [0.08, 0.32],
 'baseline_wander': True,
 'bw': True,
 'bw_fs': array([0.33, 0.1 , 0.05, 0.01]),
 'bw_ampl_ratio': array([[0.01, 0.01, 0.02, 0.03],
        [0.01, 0.02, 0.04, 0.05],
        [0.1 , 0.06, 0.04, 0.02],
        [0.02, 0.04, 0.07, 0.1 ],
        [0.05, 0.1 , 0.16, 0.25],
        [0.1 , 0.15, 0.25, 0.3 ],
        [0.25, 0.25, 0.3 , 0.35]]),


# vanilla unet

In [9]:
model_cfg = deepcopy(ModelCfg)
model_cfg.model_name = "unet"

In [10]:
config = deepcopy(TrainCfg)
config.model_name = model_cfg.model_name

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger = init_logger(log_dir=config.log_dir, verbose=2)
logger.info(f"\n{'*'*20}   Start Training   {'*'*20}\n")
logger.info(f"Using device {device}")
logger.info(f"Using torch of version {torch.__version__}")
logger.info(f"with configuration\n{dict_to_str(config)}")
# print(f"\n{'*'*20}   Start Training   {'*'*20}\n")
# print(f"Using device {device}")
# print(f"Using torch of version {torch.__version__}")
# print(f"with configuration\n{dict_to_str(config)}")

model_name = config.model_name.lower()
model_config = deepcopy(ModelCfg[model_name])

if model_name == "subtract_unet":
    model = ECG_SUBTRACT_UNET_CPSC2019(
        n_leads=config.n_leads,
        config=model_config,
    )
elif model_name == "unet":
    model = ECG_UNET_CPSC2019(
        n_leads=config.n_leads,
        config=model_config,
    )

if torch.cuda.device_count() > 1:
    model = DP(model)
#     model = DDP(model)

model.to(device=device)
model.__DEBUG__ = False

log file path: /home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/log/log_2021-06-22-13-53-00.txt
levels of c_handler and f_handler are set DEBUG
ECG-UNET - INFO - 
********************   Start Training   ********************

ECG-UNET - INFO - Using device cuda
ECG-UNET - INFO - Using torch of version 1.8.1+cu111
ECG-UNET - INFO - with configuration
{
    "fs": 500
    "db_dir": "/home/wenh06/Jupyter/data/CPSC2019/"
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/log"
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/checkpoints"
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/saved_models"
    "final_model_name": None
    "keep_checkpoint_max": 20
    "train_ratio": 0.8
    "input_len": 5000
    "classes": [
        "N"
    ]
    "n_leads": 1
    "bias_thr": 37
    "skip_dist": 250
    "label_smoothing": 0.0
    "random_normalize": T

In [12]:
model.device_ids

[0, 1]

In [13]:
best_state_dict = train(
    model=model,
    model_config=model_config,
    device=device,
    config=config,
    logger=logger,
    debug=True,
)

ECG-UNET - INFO - training configurations are as follows:
{
    "fs": 500
    "db_dir": "/home/wenh06/Jupyter/data/CPSC2019/"
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/log"
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/checkpoints"
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/saved_models"
    "final_model_name": None
    "keep_checkpoint_max": 20
    "train_ratio": 0.8
    "input_len": 5000
    "classes": [
        "N"
    ]
    "n_leads": 1
    "bias_thr": 37
    "skip_dist": 250
    "label_smoothing": 0.0
    "random_normalize": True
    "random_normalize_mean": [
        -0.05, 0.1
    ]
    "random_normalize_std": [
        0.08, 0.32
    ]
    "baseline_wander": True
    "bw": True
    "bw_fs": [0.33 0.1  0.05 0.01]
    "bw_ampl_ratio": [[0.01 0.01 0.02 0.03]
 [0.01 0.02 0.04 0.05]
 [0.1  0.06 0.04 0.02]
 [0.02 0.04 0.07 0.1 ]
 [0.05 0.1  

Epoch 1/300:  96%|███████████████████████████████████████████▏ | 1536/1600 [00:05<00:00, 592.35it/s]

QRS_acc: 0.052
Scoring complete.
QRS_acc: 0.044
Scoring complete.
ECG-UNET - INFO - 
Train epoch_1:
--------------------
train/epoch_loss:        4.068034768104553
train/qrs_score:         0.052

test/qrs_score:          0.044
---------------------------------

ECG-UNET - INFO - Checkpoint 1 saved!


Epoch 2/300:  96%|███████████████████████████████████████████▏ | 1536/1600 [00:02<00:00, 959.30it/s]

QRS_acc: 0.0695
Scoring complete.
QRS_acc: 0.058
Scoring complete.
ECG-UNET - INFO - 
Train epoch_2:
--------------------
train/epoch_loss:        3.3410195112228394
train/qrs_score:         0.0695

test/qrs_score:          0.058
---------------------------------

ECG-UNET - INFO - Checkpoint 2 saved!


...

QRS_acc: 0.9647
Scoring complete.
QRS_acc: 0.8525
Scoring complete.
ECG-UNET - INFO - 
Train epoch_86:
--------------------
train/epoch_loss:        1.6162223517894745
train/qrs_score:         0.9647

test/qrs_score:          0.8525
---------------------------------

ECG-UNET - INFO - Checkpoint 86 saved!


Epoch 87/300:  96%|██████████████████████████████████████████▏ | 1536/1600 [00:02<00:00, 958.72it/s]

QRS_acc: 0.9598
Scoring complete.
QRS_acc: 0.8632
Scoring complete.
ECG-UNET - INFO - 
Train epoch_87:
--------------------
train/epoch_loss:        1.6083958148956299
train/qrs_score:         0.9598

test/qrs_score:          0.8632
---------------------------------

ECG-UNET - INFO - early stopping is triggered at epoch 87


Epoch 87/300: 100%|████████████████████████████████████████████| 1600/1600 [00:06<00:00, 263.63it/s]

ECG-UNET - INFO - Best model saved to /home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/saved_models/ECG_UNET_CPSC2019_epoch_2021-06-22-14-02-00_BestModel_challenge_score(qrs_score)_0.8688.pth.tar!


In [ ]:
# best_state_dict

In [ ]:
# best_model = ECG_UNET_CPSC2019(
#     n_leads=config.n_leads,
#     config=model_config,
# )

In [ ]:
# best_model.load_state_dict(best_state_dict)

In [ ]:
# best_model

# CPSC2019 subtract unet

In [14]:
model_cfg = deepcopy(ModelCfg)
model_cfg.model_name = "subtract_unet"

config = deepcopy(TrainCfg)
config.model_name = model_cfg.model_name

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger = init_logger(log_dir=config.log_dir, verbose=2)
logger.info(f"\n{'*'*20}   Start Training   {'*'*20}\n")
logger.info(f"Using device {device}")
logger.info(f"Using torch of version {torch.__version__}")
logger.info(f"with configuration\n{dict_to_str(config)}")
# print(f"\n{'*'*20}   Start Training   {'*'*20}\n")
# print(f"Using device {device}")
# print(f"Using torch of version {torch.__version__}")
# print(f"with configuration\n{dict_to_str(config)}")

model_name = config.model_name.lower()
model_config = deepcopy(ModelCfg[model_name])

if model_name == "subtract_unet":
    model = ECG_SUBTRACT_UNET_CPSC2019(
        n_leads=config.n_leads,
        config=model_config,
    )
elif model_name == "unet":
    model = ECG_UNET_CPSC2019(
        n_leads=config.n_leads,
        config=model_config,
    )

if torch.cuda.device_count() > 1:
    model = DP(model)
#     model = DDP(model)

model.to(device=device)
model.__DEBUG__ = False

log file path: /home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/log/log_2021-06-22-14-03-09.txt
levels of c_handler and f_handler are set DEBUG
ECG-UNET - INFO - 
********************   Start Training   ********************

ECG-UNET - INFO - 
********************   Start Training   ********************

ECG-UNET - INFO - Using device cuda
ECG-UNET - INFO - Using device cuda
ECG-UNET - INFO - Using torch of version 1.8.1+cu111
ECG-UNET - INFO - Using torch of version 1.8.1+cu111
ECG-UNET - INFO - with configuration
{
    "fs": 500
    "db_dir": "/home/wenh06/Jupyter/data/CPSC2019/"
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/log"
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/checkpoints"
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/saved_models"
    "final_model_name": None
    "keep_checkpoint_max": 20
    "train_ratio": 

In [16]:
best_state_dict = train(
    model=model,
    model_config=model_config,
    device=device,
    config=config,
    logger=logger,
    debug=True,
)

ECG-UNET - INFO - training configurations are as follows:
{
    "fs": 500
    "db_dir": "/home/wenh06/Jupyter/data/CPSC2019/"
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/log"
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/checkpoints"
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/saved_models"
    "final_model_name": None
    "keep_checkpoint_max": 20
    "train_ratio": 0.8
    "input_len": 5000
    "classes": [
        "N"
    ]
    "n_leads": 1
    "bias_thr": 37
    "skip_dist": 250
    "label_smoothing": 0.0
    "random_normalize": True
    "random_normalize_mean": [
        -0.05, 0.1
    ]
    "random_normalize_std": [
        0.08, 0.32
    ]
    "baseline_wander": True
    "bw": True
    "bw_fs": [0.33 0.1  0.05 0.01]
    "bw_ampl_ratio": [[0.01 0.01 0.02 0.03]
 [0.01 0.02 0.04 0.05]
 [0.1  0.06 0.04 0.02]
 [0.02 0.04 0.07 0.1 ]
 [0.05 0.1  

Epoch 1/300: 100%|████████████████████████████████████████████| 1600/1600 [00:01<00:00, 1292.28it/s]

QRS_acc: 0.0486
Scoring complete.
QRS_acc: 0.0435
Scoring complete.
ECG-UNET - INFO - 
Train epoch_1:
--------------------
train/epoch_loss:        4.2473196387290955
train/qrs_score:         0.0486

test/qrs_score:          0.0435
---------------------------------

ECG-UNET - INFO - 
Train epoch_1:
--------------------
train/epoch_loss:        4.2473196387290955
train/qrs_score:         0.0486

test/qrs_score:          0.0435
---------------------------------

ECG-UNET - INFO - Checkpoint 1 saved!
ECG-UNET - INFO - Checkpoint 1 saved!


Epoch 2/300: 100%|████████████████████████████████████████████| 1600/1600 [00:01<00:00, 1312.14it/s]

QRS_acc: 0.2052
Scoring complete.
QRS_acc: 0.2112
Scoring complete.
ECG-UNET - INFO - 
Train epoch_2:
--------------------
train/epoch_loss:        3.5186086297035217
train/qrs_score:         0.2052

test/qrs_score:          0.2112
---------------------------------

ECG-UNET - INFO - 
Train epoch_2:
--------------------
train/epoch_loss:        3.5186086297035217
train/qrs_score:         0.2052

test/qrs_score:          0.2112
---------------------------------

ECG-UNET - INFO - Checkpoint 2 saved!
ECG-UNET - INFO - Checkpoint 2 saved!


...

QRS_acc: 0.9631
Scoring complete.
QRS_acc: 0.887
Scoring complete.
ECG-UNET - INFO - 
Train epoch_95:
--------------------
train/epoch_loss:        1.5823836624622345
train/qrs_score:         0.9631

test/qrs_score:          0.887
---------------------------------

ECG-UNET - INFO - 
Train epoch_95:
--------------------
train/epoch_loss:        1.5823836624622345
train/qrs_score:         0.9631

test/qrs_score:          0.887
---------------------------------

ECG-UNET - INFO - Checkpoint 95 saved!
ECG-UNET - INFO - Checkpoint 95 saved!


Epoch 96/300: 100%|███████████████████████████████████████████| 1600/1600 [00:01<00:00, 1331.63it/s]

QRS_acc: 0.9701
Scoring complete.
QRS_acc: 0.8998
Scoring complete.
ECG-UNET - INFO - 
Train epoch_96:
--------------------
train/epoch_loss:        1.5784481018781662
train/qrs_score:         0.9701

test/qrs_score:          0.8998
---------------------------------

ECG-UNET - INFO - 
Train epoch_96:
--------------------
train/epoch_loss:        1.5784481018781662
train/qrs_score:         0.9701

test/qrs_score:          0.8998
---------------------------------

ECG-UNET - INFO - early stopping is triggered at epoch 96
ECG-UNET - INFO - early stopping is triggered at epoch 96


Epoch 96/300: 100%|████████████████████████████████████████████| 1600/1600 [00:04<00:00, 362.38it/s]

ECG-UNET - INFO - Best model saved to /home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/saved_models/ECG_SUBTRACT_UNET_CPSC2019_epoch_2021-06-22-14-10-34_BestModel_challenge_score(qrs_score)_0.9028.pth.tar!
ECG-UNET - INFO - Best model saved to /home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_unet_cpsc2019/saved_models/ECG_SUBTRACT_UNET_CPSC2019_epoch_2021-06-22-14-10-34_BestModel_challenge_score(qrs_score)_0.9028.pth.tar!


# DEBUG

In [ ]:
from train.database_reader.database_reader.cpsc_databases.cpsc2019 import CPSC2019 as DR
from dataset import CPSC2019 as DG

In [ ]:
dr = DR(db_dir=TrainCfg.db_dir)

In [ ]:
dg = DG(TrainCfg)

In [ ]:
dg.__DEBUG__ = False

In [ ]:
values, labels = dg[2]

In [ ]:
labels.shape

In [ ]:
plt.plot(labels[...,0])

In [ ]:
# for s,l in dg:
for idx in range(len(dg)):
    s,l = dg[idx]
    fig, ax = plt.subplots()
    ax.plot(l[...,0])
    plt.show()
    print("-"*50)

In [ ]:
values = values.astype(np.float32)

In [ ]:
# labels.shape

In [ ]:
# ?best_model.inference

In [ ]:
pred, rpeaks = best_model.inference(values[np.newaxis,...])

In [ ]:
np.where(pred>0.5)

In [ ]:
pred.shape, values.shape

In [ ]:
fig, ax = plt.subplots(figsize=(20,4))
ax.plot(values[0])
ax.plot(pred[0]-0.5,c="red")
plt.show()

In [ ]:
hehe_fwd = best_model.forward(torch.from_numpy(values[np.newaxis,...]).to(device))

In [ ]:
plt.plot(hehe_fwd.squeeze().cpu().detach().numpy())

In [ ]:
?nn.Sigmoid

In [ ]:
plt.plot(labels[0])

In [ ]:
np.unique(labels[0])